# Uniview module for LMC stars

*Aaron Geller, 2018*

### Imports and function definitions

In [19]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [54]:
Template = """mesh
{  
    data LMC    ./modules/$folderName/LMC.raw
    dataHints LMC disablePostprocessing

    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f coronaRad 10  | public | desc "size of stellar corona" | widget slider | range 0 50
            vec1f coronaFac 1.2  | public | desc "weight of noise" | widget slider | range 0 2
            vec1f Alpha 1.  | public | desc "overall alpha" | widget slider | range 0 1
        
            
        }
    }
    
    ############# stellar corona for LMC stars   
    pass
    {
        useDataObject LMC
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/corona.gs
                vertexShader ./modules/$folderName/corona.vs
                fragmentShader   ./modules/$folderName/corona.fs
                texture bb ./modules/$folderName/bb.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }
                
                stateManagerVar __objectName__.coronaRad  coronaRad
                stateManagerVar __objectName__.coronaFac  coronaFac
                stateManagerVar __objectName__.Alpha  Alpha

                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }


}"""

### LMC class

In [55]:
class LMC():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("corona.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("corona.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("corona.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [56]:
model = LMC(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [57]:
scene.setname("LMCGaia")
scene.setparent("MilkyWay")
scene.setunit(3.0856775814671916e+16) #what should I use for the units here? My data are in parsecs
scene.setentrydist(100000.)
scene.setstaticposition(1943.83535919, 18089.69093062, -48072.2850105)

model.object.setcameraradius(0.01)
model.object.setcoord(scene.name)
model.object.setname("LMCGaia")
model.object.setguiname("/KavliLecture/Larson/LMCGaia")
model.object.settargetradius(10)
model.object.showatstartup(False)
model.cr = 100000

modinfo.setname("LMCGaia")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Shane Larson<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Northwestern University")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for LMC based on Gaia data")
#modinfo.setthumbnail("data/R0010133.JPG")
modinfo.setversion("1.0")

generator.generate("LMCGaia",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [49]:
uvmod.Utility.senduvcommand(model.object.name+".reload")

*Copy modified Shader files and reload*

In [9]:
from config import Settings

In [99]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## SN1987a position 

*From the [wikipedia page](https://en.wikipedia.org/wiki/SN_1987A)*

In [17]:
from astropy.coordinates import SkyCoord
from astropy import units

In [26]:
RA = "05h 35m 28.03s" #right ascension
Dec= "−69d 16m 11.79s" #declination
dist = (51.4 *units.kpc).to(units.pc) #distance

coord = SkyCoord(RA, Dec, dist)

print(coord.cartesian)

print((1.*units.pc).to(units.m))

(1943.83535919, 18089.69093062, -48072.2850105) pc
3.0856775814671916e+16 m


## Center of LMC  

*From the [wikipedia page](https://en.wikipedia.org/wiki/Large_Magellanic_Cloud)*

In [2]:
from astropy.coordinates import SkyCoord
from astropy import units

In [3]:
RA = "05h 23m 34.5s" #right ascension
Dec= "−69d 45m 22s" #declination
dist = (50 *units.kpc).to(units.pc) #distance

coord = SkyCoord(RA, Dec, dist)

print(coord.cartesian)

(2738.1323679, 17082.79901096, -46911.41235443) pc
